In [2]:
#Procedimiento realizado a la tabla producto
#   1.Se borran los datos repetidos si existen y los valores null
#   2.Se modifica el id de la tabla para lograr que sea un int

#Se carga las librerias a usar
import pandas as pd

#Se carga la tabla desde el archivo
df_producto = pd.read_parquet("Datasets\producto.parquet")

# Se borran valores repetidos
df_producto.drop_duplicates(inplace=True)

#Se borran valores nulos pero sin tomar en cuenta categorias
df_producto.dropna(subset=['id','marca','nombre','presentacion'],inplace=True)
#print(df_producto.info())

#Se verifica el id  creando lista para la columna con valores Id 
# que se puedan convertir en int 
id_producto =[]
for i in range(len(df_producto['id'])):
    try:
        id_producto.append(int(df_producto.iloc[i,0]))
    except ValueError:
        id_producto.append(None)

#Se agrega la columna ID_producto con valores de id int
df_producto.insert(loc = 0 , column = "Id_producto", value = id_producto)

#Se guardan valores con problemas, con mas tiempo se pueden mejorar e integrar a la tabla
df_erroresId = df_producto[df_producto["Id_producto"].isnull()].drop(["Id_producto"], axis=1)

#Se borran valores nulos
df_producto.dropna(subset="Id_producto",inplace=True)

#Se elimina la columna Id
df_producto.drop(["id"], axis=1,inplace=True)



#Se exporta la base de datos a MySQL
import sqlalchemy as db
database_username='root'            # Nombre de cliente en MySQL Workbrench
database_password='JCSR.MySQL123'   # Contraseña de MySQL Workbrench
database_ip='localhost'
database_name='ETL_Lab01'           # Nombre de Base de datos a donde nos conectaremos (se debio crear primero en Mysql)
database_conection = db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}',
                                    encoding='latin1')
coneccion=database_conection.connect()
df_producto.to_sql('producto',coneccion, index=False)

import pymysql
conexion = pymysql.connect(
   host = 'localhost',
   user = 'root',
   passwd='JCSR.MySQL123',
   db='ETL_Lab01'           #LA base de datos se debio crear primero en Mysql
)
cursor = conexion.cursor()
try:
    #Se borra la base de datos si existe para cargarla nuevamente
    cursor.execute("alter table producto add primary key (Id_producto)")
finally:
    conexion.close()
print("Carga de tabla 'producto' a MySQL completa")

Carga de tabla 'producto' a MySQL completa
